# Extract for our supposed ETL

In [2]:
import requests
from pprint import pprint

def get_request(code):
    URL = "https://api.company-information.service.gov.uk/company/{}".format(code)
    headers = {'Authorization': "0365de33-ebf5-43d0-b11d-16e9cb202611"}
    status = requests.get(URL, headers=headers)
    return status


code = "10499932"
status = get_request(code)

In [3]:
code = 10499931
status_list = []
for i in range(10):
    code += 1
    status_list.append(get_request(code))

SSLError: HTTPSConnectionPool(host='api.company-information.service.gov.uk', port=443): Max retries exceeded with url: /company/10499937 (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:997)')))

In [9]:


for status in status_list:
    pprint(status.json())
    our_test_json = status.json()
    break
    
"accounts.accounting_reference_date"

{'accounts': {'accounting_reference_date': {'day': '31', 'month': '12'},
              'last_accounts': {'made_up_to': '2021-12-31',
                                'period_end_on': '2021-12-31',
                                'period_start_on': '2021-01-01',
                                'type': 'total-exemption-full'},
              'next_accounts': {'due_on': '2023-09-30',
                                'overdue': False,
                                'period_end_on': '2022-12-31',
                                'period_start_on': '2022-01-01'},
              'next_due': '2023-09-30',
              'next_made_up_to': '2022-12-31',
              'overdue': False},
 'can_file': True,
 'company_name': 'YOUNG FOODIES LIMITED',
 'company_number': '10499932',
 'company_status': 'active',
 'confirmation_statement': {'last_made_up_to': '2022-09-22',
                            'next_due': '2023-10-06',
                            'next_made_up_to': '2023-09-22',
                      

'accounts.accounting_reference_date'

In [4]:
status_list[0]

<Response [200]>

In [5]:
response_list = []
for status in status_list:
    response_list.append(status.json())


# Transform

In [12]:
pprint(list(our_test_json.keys()))

['etag',
 'links',
 'company_number',
 'confirmation_statement',
 'company_name',
 'sic_codes',
 'previous_company_names',
 'undeliverable_registered_office_address',
 'accounts',
 'has_charges',
 'has_insolvency_history',
 'company_status',
 'jurisdiction',
 'type',
 'date_of_creation',
 'registered_office_address',
 'registered_office_is_in_dispute',
 'has_super_secure_pscs',
 'can_file']


In [13]:
for k in our_test_json.keys():
    print(k)
    
print("\n\n")

for v in our_test_json.values():
    print(v)
    
print("\n\n")

for k, v in our_test_json.items():
    print(k)
    if type(v) is dict:
        print(list(v.keys()))
    print("\n")

etag
links
company_number
confirmation_statement
company_name
sic_codes
previous_company_names
undeliverable_registered_office_address
accounts
has_charges
has_insolvency_history
company_status
jurisdiction
type
date_of_creation
registered_office_address
registered_office_is_in_dispute
has_super_secure_pscs
can_file



4f6d6e2a8a801eda687e4eba0b6420e0f279a856
{'persons_with_significant_control': '/company/10499932/persons-with-significant-control', 'officers': '/company/10499932/officers', 'filing_history': '/company/10499932/filing-history', 'self': '/company/10499932'}
10499932
{'next_made_up_to': '2023-09-22', 'next_due': '2023-10-06', 'overdue': False, 'last_made_up_to': '2022-09-22'}
YOUNG FOODIES LIMITED
['70229']
[{'name': 'HOUSE FOOD LIMITED', 'effective_from': '2016-11-28', 'ceased_on': '2017-02-28'}]
False
{'last_accounts': {'made_up_to': '2021-12-31', 'type': 'total-exemption-full', 'period_end_on': '2021-12-31', 'period_start_on': '2021-01-01'}, 'next_accounts': {'period_en

In [5]:
# We want to transform the data into mostly an unnested output!

# If our dict has a value which is also a dict, then we want to extract the important keys, and their values from that nested dict

my_dict = {"location": {
    "id": 3672652,
    "name": "gsgsuygd", 
    "people_associated": 20, 
    "country": "gsjaygjsd"
},
"id": 267276,
"idn": 356252}

print(my_dict.keys())

dict_keys(['location', 'id', 'idn'])


In [15]:
"""INSERT INTO table_name VALUES(id, idn, ...)""" # for SQL based DBs databases

'INSERT INTO table_name VALUES(id, idn, ...)'

In [16]:
{"location": {"id": 3672652,
             "name": "gsgsuygd"},
"id": 267276,
"idn": 356252}

{'location': {'id': 3672652, 'name': 'gsgsuygd'}, 'id': 267276, 'idn': 356252}

In [17]:
# for dictionaries 
"location.id"
{"location": [5625453, 25726536, 235657]}

# Pandas (dataframing tool used for datamodelling)
# PySpark (dataframing tool used for datamodelling integrated with SQL in Cloud based/distributed systems)

{'location': [5625453, 25726536, 235657]}

In [26]:
def str_replace_with_empty(string, replacement_list):
    for obj in replacement_list: 
        string = string.replace(obj, '')
    return string
    
    
def extract_key_value (dct, key):
    try:
        return dct.get(key)
    except Exception as e:
        return False
    

def extract_nested_key_value (inpDict, nested_key):
    dct = inpDict

    nested_key_list = nested_key.split('.')

    for colName in nested_key_list:
        dct = extract_key_value (dct, colName)
        if not dct:
            return None

    return dct


def unnested_key_value_extract (inpDict, unnested_key):
    dct = inpDict

    key_list = unnested_key.split('.')
    nested_key_list = key_list[:-1]
    unnest_key = key_list[-1]

    for colName in nested_key_list:
        dct = extract_key_value (dct, colName)
        if not dct:
            return None
    
    
def transformations_on_batch(inpBatch, map_proj_list):
        mapped_batch = []
        input_proj_list = sorted(map_proj_list, key=lambda x: x['index'])
        for rec in inpBatch:

            mapped_dict = {}
            for colMap in input_proj_list:
                colName, colAlias = colMap['colName'], colMap['alias']
                each_obj = rec.get(colMap['colName'].split(".")[0])
                if each_obj != None:
                    col_type = type(each_obj)
                else:
                    col_type = None
                map_type = colMap['map_type']

                # handling for datetime object 
                if (f"{col_type}" == "<class 'datetime.date'>") or (f"{col_type}" == "<class 'datetime.datetime'>") or (f"{col_type}" == "<class 'datetime.time'>"):

                    # convert datetime to isoformat
                    rec[colName] = each_obj.isoformat()
                    
                # any other datatype
                
                else:
                    pass

                # handling for jsons, alias, and col name        
                if map_type == "None":
                    mapped_dict[colName] = extract_nested_key_value (rec, colName)

                else:
                    mapped_dict[colName] = unnested_key_value_extract (rec, colName)

                # handling string notation errors
                if type(mapped_dict[colName]) == str:
                    replacement_list = ['\r', '\n', '<p>', '</p>', '*', '<br>']
                    
                    mapped_dict[colName] = str_replace_with_empty(mapped_dict[colName], replacement_list)
                    
                # handling for alias, and col name
                if colAlias != "None":
                    alias = colAlias.replace(".", "_")
                    mapped_dict[alias] = mapped_dict[colName]
                    

                    # if we CHOOSE to replace the column in the dictionary
                    mapped_dict.pop(colName)
                else:
                    name = colName.replace(".", "_")
                    mapped_dict[name] = mapped_dict[colName]
                    
                    if "." in colName:
                        mapped_dict.pop(colName)
                    

                
                
            mapped_batch.append(mapped_dict)
        return mapped_batch


In [27]:
our_mapping_config = [
    {"index": 1, "colName": "etag", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 2, "colName": "jurisdiction", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 3, "colName": "accounts.next_due", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 4, "colName": "links.filing_history", "type": "str", "alias": "None", "map_type": "None"}
]

transformed_data = transformations_on_batch(response_list, our_mapping_config)

In [28]:
transformed_data

[{'etag': '4f6d6e2a8a801eda687e4eba0b6420e0f279a856',
  'jurisdiction': 'england-wales',
  'accounts_next_due': '2023-09-30',
  'links_filing_history': '/company/10499932/filing-history'},
 {'etag': 'dfac00ef4fd3bb25044c48e8cc56cc3a93ea5101',
  'jurisdiction': 'england-wales',
  'accounts_next_due': None,
  'links_filing_history': '/company/10499933/filing-history'},
 {'etag': 'b9dab31503f80c4a13f6b4ca04de01bf2b4ff4a2',
  'jurisdiction': 'england-wales',
  'accounts_next_due': None,
  'links_filing_history': '/company/10499934/filing-history'},
 {'etag': 'a11b721118c058615f6ab4cff12023347a5c2ad0',
  'jurisdiction': 'england-wales',
  'accounts_next_due': None,
  'links_filing_history': '/company/10499935/filing-history'},
 {'etag': 'ac526574fb40015161277eee0a0a276d6a7a5106',
  'jurisdiction': 'england-wales',
  'accounts_next_due': None,
  'links_filing_history': '/company/10499936/filing-history'}]

In [29]:
import pandas as pd

df = pd.DataFrame(transformed_data)

df

,etag,jurisdiction,accounts_next_due,links_filing_history
0,4f6d6e2a8a801eda687e4eba0b6420e0f279a856,england-wales,2023-09-30,/company/10499932/filing-history
1,dfac00ef4fd3bb25044c48e8cc56cc3a93ea5101,england-wales,None,/company/10499933/filing-history
2,b9dab31503f80c4a13f6b4ca04de01bf2b4ff4a2,england-wales,None,/company/10499934/filing-history
3,a11b721118c058615f6ab4cff12023347a5c2ad0,england-wales,None,/company/10499935/filing-history
4,ac526574fb40015161277eee0a0a276d6a7a5106,england-wales,None,/company/10499936/filing-history


In [30]:
df['jurisdiction']

0    england-wales
1    england-wales
2    england-wales
3    england-wales
4    england-wales
Name: jurisdiction, dtype: object

In [33]:
df['jurisdiction'].iloc[0:2]

0    england-wales
1    england-wales
Name: jurisdiction, dtype: object

In [32]:
df['jurisdiction'].iloc[0:2].values

array(['england-wales', 'england-wales'], dtype=object)

In [ ]:
company_name': 'YOUNG FOODIES LIMITED',
 'company_number': '10499932',
 'company_status': 'active',
 'confirmation_statement': {'last_made_up_to': '2022-09-22',
                            'next_due': '2023-10-06',
                            'next_made_up_to': '2023-09-22',
                            'overdue': False},
 'date_of_creation': '2016-11-28',
 'etag': '4f6d6e2a8a801eda687e4eba0b6420e0f279a856',
 'has_charges': False,
 'has_insolvency_history': False,
 'has_super_secure_pscs': False,
 'jurisdiction': 'england-wales',
 'links': {'filing_history': '/company/10499932/filing-history',
           'officers': '/company/10499932/officers',
           'persons_with_significant_control': '/company/10499932/persons-with-significant-control',
           'self': '/company/10499932'},
 'previous_company_names': [{'ceased_on': '2017-02-28',
                             'effective_from': '2016-11-28',
                             'name': 'HOUSE FOOD LIMITED'}],
 'registered_office_address': {'address_line_1': 'Room 2.14 New Covent Garden '
                                                 'Market',
                               'country': 'England',
                               'locality': 'London',
                               'postal_code': 'SW8 5EL'},

In [34]:
our_mapping_config = [
    {"index": 1, "colName": "etag", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 2, "colName": "jurisdiction", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 3, "colName": "accounts.next_due", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 4, "colName": "links.filing_history", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 5, "colName": "date_of_creation", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 6, "colName": "company_number", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 7, "colName": "registered_office_address.address_line_1", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 8, "colName": "links.persons_with_significant_control", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 9, "colName": "confirmation_statement.next_due", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 10, "colName": "company_status", "type": "str", "alias": "None", "map_type": "None"},
    {"index": 11, "colName": "registered_office_address.postal_code", "type": "str", "alias": "None", "map_type": "None"}
]

transformed_data = transformations_on_batch(response_list, our_mapping_config)

In [35]:
transformed_data

[{'etag': '4f6d6e2a8a801eda687e4eba0b6420e0f279a856',
  'jurisdiction': 'england-wales',
  'accounts_next_due': '2023-09-30',
  'links_filing_history': '/company/10499932/filing-history',
  'date_of_creation': '2016-11-28',
  'company_number': '10499932',
  'registered_office_address_address_line_1': 'Room 2.14 New Covent Garden Market',
  'links_persons_with_significant_control': '/company/10499932/persons-with-significant-control',
  'confirmation_statement_next_due': '2023-10-06',
  'company_status': 'active',
  'registered_office_address_postal_code': 'SW8 5EL'},
 {'etag': 'dfac00ef4fd3bb25044c48e8cc56cc3a93ea5101',
  'jurisdiction': 'england-wales',
  'accounts_next_due': None,
  'links_filing_history': '/company/10499933/filing-history',
  'date_of_creation': '2016-11-28',
  'company_number': '10499933',
  'registered_office_address_address_line_1': '89a Hermitage Road',
  'links_persons_with_significant_control': '/company/10499933/persons-with-significant-control',
  'confirmat

In [36]:
import pandas as pd

df = pd.DataFrame(transformed_data)

df

,etag,jurisdiction,accounts_next_due,links_filing_history,date_of_creation,company_number,registered_office_address_address_line_1,links_persons_with_significant_control,confirmation_statement_next_due,company_status,registered_office_address_postal_code
0,4f6d6e2a8a801eda687e4eba0b6420e0f279a856,england-wales,2023-09-30,/company/10499932/filing-history,2016-11-28,10499932,Room 2.14 New Covent Garden Market,/company/10499932/persons-with-significant-con...,2023-10-06,active,SW8 5EL
1,dfac00ef4fd3bb25044c48e8cc56cc3a93ea5101,england-wales,None,/company/10499933/filing-history,2016-11-28,10499933,89a Hermitage Road,/company/10499933/persons-with-significant-con...,None,dissolved,N4 1LU
2,b9dab31503f80c4a13f6b4ca04de01bf2b4ff4a2,england-wales,None,/company/10499934/filing-history,2016-11-28,10499934,300 Hale Road,/company/10499934/persons-with-significant-con...,None,dissolved,WA15 8SP
3,a11b721118c058615f6ab4cff12023347a5c2ad0,england-wales,None,/company/10499935/filing-history,2016-11-28,10499935,40 Kingsland Crescent,/company/10499935/persons-with-significant-con...,None,dissolved,L11 7AP
4,ac526574fb40015161277eee0a0a276d6a7a5106,england-wales,None,/company/10499936/filing-history,2016-11-28,10499936,Vitkin Farm Grange Road,None,None,dissolved,CO5 0UH


# Load

# Testing DFM API

In [12]:
import requests
from pprint import pprint



def DFM_fetch_request():

    URL = "https://marketwatch.dfm.ae/api/fetch"
    # payload = [{"id":"1d77991a-f9a0-1177-1cc2-e016575903a6","command":"sitefinity_securities","data":[{"id":"language","value":"en"}]},{"id":"c6690c71-11fb-606b-394e-962b1e5a9392","command":"news","data":[{"id":"language","value":"en"}]},{"id":"22d72cd9-5c57-35dc-a08d-7c5f8fa8183c","command":"sitefinity_sectors","data":[{"id":"language","value":"en"}]},{"id":"0474b844-589e-c81a-6917-387b08c3c64a","command":"sitefinity_exchanges","data":[{"id":"language","value":"en"}]},{"id":"865fe490-0737-d3f2-72ac-2ec488b230e5","command":"sitefinity_sectors","data":[{"id":"language","value":"en"}]}]
    payload = [{"id":"1adc4bec-825b-ea31-070b-22ed849d2ec5","command":"stocks","data":[{"id":"lastrefreshtime","value":"0"},{"id":"id","value":""}]}]
    headers = {
        "Accept-Language": "en-US,en;q=0.9",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Content-Length": "577",
        "Content-Type": "application/json",
        "Cookie": "ASP.NET_SessionId=c3stqakyvpwdzvj1haqpldub; marketcookiepers=ffffffffaf1c0d1a45525d5f4f58455e445a4a423660; _ga=GA1.3.853830670.1679915722; _gid=GA1.3.1682684732.1679915722; _gat=1; dimensions=",
        "Host": "marketwatch.dfm.ae",
        "Origin": "https://marketwatch.dfm.ae",
        "Pragma": "no-cache",
        "Referer": "https://marketwatch.dfm.ae/",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"
    }

    return requests.post(URL, headers=headers, json=payload)



In [13]:
status = DFM_fetch_request()

print("The status code of the Request is: {}\n".format(status))

The status code of the Request is: <Response [200]>



In [14]:
response = status.json()

In [23]:
for row in response[0]['data']:
    pprint(row)
    break

{'adjusted': None,
 'averageprice': 0.208,
 'biddecimal': 3.0,
 'bidprice': 0.209,
 'bidvolume': 134314,
 'capital': None,
 'changepercentage': 1.449,
 'closingprice': 0.21,
 'contractsize': 0.0,
 'creationdate': None,
 'disclosures': None,
 'highestin52weeks': 0.31,
 'highestprice': 0.211,
 'highestpricetime': '2023-03-27T12:02:35',
 'id': 'UPP',
 'indicative_volume': 0,
 'indicativeprice': 0.0,
 'indicativevolume': 0,
 'lastraded': '2023-03-27T14:43:21',
 'lastradeprice': 0.21,
 'lastradetime': '2023-03-27T14:43:21',
 'lastradevolume': 35000,
 'lastsessionprice': 0.21,
 'lowestin52weeks': 0.205,
 'lowestprice': 0.206,
 'lowestpricetime': '2023-03-27T10:13:13',
 'market': '510',
 'maturitydate': None,
 'modificationdate': '2023-03-27T15:00:15',
 'name': None,
 'netchange': 0.003,
 'offerdecimal': 3.0,
 'offerprice': 0.21,
 'offervolume': 286309,
 'open_interest': 0.0,
 'openingprice': 0.207,
 'previousclosedate': '2023-03-24T00:00:00',
 'previousclosingprice': 0.207,
 'rate': 1.0,
 's

In [24]:
import pandas as pd

df = pd.DataFrame(response[0]['data'])
df.to_csv("DFM_api_data.csv")

In [13]:
import pandas as pd

pprint(dir(pd))
pprint(dir(pd.ArrowDtype))

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__from_arrow__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_can_hold_na',
 '_get_common_dtype',
 '_is_boolean',
 '_is_numeric',
 '_metadata',
 'construct_array_type',
 'construct_from_string',
 'empty',
 'is_dtype',
 'itemsize',
 'kind',
 'na_value',
 'name',
 'names',
 'numpy_dtype',
 'type']


In [8]:
import pandas as pd


df = pd.read_csv("precoro_item-wise_monthly_avg_price.csv")

df

,Unnamed: 0,created_date_time_month,created_date_time,location_unique_code,item_name,avg_item_price
0,4751,2022-11-01,Nov-2022,RIY001,/Lehlah fish fillet one carton as a sample,12.00
1,2815,2022-08-01,Aug-2022,RYI004,1 Oz Plastic Container 25x100,140.00
2,3900,2022-10-01,Oct-2022,RIY001,1 Oz Plastic Container 25x100,140.00
3,3901,2022-10-01,Oct-2022,RIY001,1 pcs doorbell,62.15
4,3902,2022-10-01,Oct-2022,RIY001,1 pcs laminating pouches,220.25
...,...,...,...,...,...,...
5535,3896,2022-10-01,Oct-2022,JEDRAJ001,yoghurt low fat 2kg,12.61
5536,3897,2022-10-01,Oct-2022,JEDRAJ001,zebra cup,13.04
5537,3898,2022-10-01,Oct-2022,JEDRAJ001,zk fingerprint device,1300.00
5538,3899,2022-10-01,Oct-2022,JEDRAJ001,zk uface 800 ID multi biometric and access con...,950.00


In [7]:
df[df['created_date_time_month'] == "2022-11-01"]

,Unnamed: 0,created_date_time_month,created_date_time,location_unique_code,item_name,avg_item_price
0,4751,2022-11-01,Nov-2022,RIY001,/Lehlah fish fillet one carton as a sample,12.0000
15,4752,2022-11-01,Nov-2022,RIY001,1/8 Measuring Cup,7.0000
30,4753,2022-11-01,Nov-2022,RIY001,12&16oz FLAT LID FOR PET DELI CONTAINER 1x500pcs,105.0000
31,5160,2022-11-01,Nov-2022,RYI004,12&16oz FLAT LID FOR PET DELI CONTAINER 1x500pcs,105.0000
42,4754,2022-11-01,Nov-2022,RIY001,1521 BAN INSERT PAN POLYCARBONATE 2/1-15 CM,280.0000
...,...,...,...,...,...,...
5507,4747,2022-11-01,Nov-2022,JEDRAJ001,water filter HD italy,10.0000
5512,4748,2022-11-01,Nov-2022,JEDRAJ001,whipping cream,20.0275
5522,5159,2022-11-01,Nov-2022,RIY001,wireless mouse,129.5600
5528,4749,2022-11-01,Nov-2022,JEDRAJ001,yoghurt 2kg,12.8050


# CSV transformations

In [7]:
import pandas as pd

In [8]:
df_csv = pd.read_csv("Pokemon_full.csv")

In [9]:
df_csv.shape

(890, 12)

In [14]:
df_csv.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
885    False
886    False
887    False
888    False
889    False
Length: 890, dtype: bool

In [10]:
len(df_csv.columns)

12

In [11]:
df_csv["name"].unique()

array(['bulbasaur', 'ivysaur', 'venusaur', 'charmander', 'charmeleon',
       'charizard', 'squirtle', 'wartortle', 'blastoise', 'caterpie',
       'metapod', 'butterfree', 'weedle', 'kakuna', 'beedrill', 'pidgey',
       'pidgeotto', 'pidgeot', 'rattata', 'raticate', 'spearow', 'fearow',
       'ekans', 'arbok', 'pikachu', 'raichu', 'sandshrew', 'sandslash',
       'nidoran-f', 'nidorina', 'nidoqueen', 'nidoran-m', 'nidorino',
       'nidoking', 'clefairy', 'clefable', 'vulpix', 'ninetales',
       'jigglypuff', 'wigglytuff', 'zubat', 'golbat', 'oddish', 'gloom',
       'vileplume', 'paras', 'parasect', 'venonat', 'venomoth', 'diglett',
       'dugtrio', 'meowth', 'persian', 'psyduck', 'golduck', 'mankey',
       'primeape', 'growlithe', 'arcanine', 'poliwag', 'poliwhirl',
       'poliwrath', 'abra', 'kadabra', 'alakazam', 'machop', 'machoke',
       'machamp', 'bellsprout', 'weepinbell', 'victreebel', 'tentacool',
       'tentacruel', 'geodude', 'graveler', 'golem', 'ponyta', 'rapida

In [12]:
df_csv

,name,pokedex id,height,weight,type,secundary type,hp,attack,defense,sp atk,sp def,speed
0,bulbasaur,1,7,69,grass,poison,45,49,49,65,65,45
1,ivysaur,2,10,130,grass,poison,60,62,63,80,80,60
2,venusaur,3,20,1000,grass,poison,80,82,83,100,100,80
3,charmander,4,6,85,fire,None,39,52,43,60,50,65
4,charmeleon,5,11,190,fire,None,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...,...,...,...
885,drakloak,886,14,110,dragon,ghost,68,80,50,60,50,102
886,dragapult,887,30,500,dragon,ghost,88,120,75,100,75,142
887,zacian,888,28,1100,fairy,None,92,130,115,80,115,138
888,zamazenta,889,29,2100,fighting,None,92,130,115,80,115,138


In [15]:
df_csv[df_csv['type'].duplicated()]

,name,pokedex id,height,weight,type,secundary type,hp,attack,defense,sp atk,sp def,speed
1,ivysaur,2,10,130,grass,poison,60,62,63,80,80,60
2,venusaur,3,20,1000,grass,poison,80,82,83,100,100,80
4,charmeleon,5,11,190,fire,None,58,64,58,80,65,80
5,charizard,6,17,905,fire,flying,78,84,78,109,85,100
7,wartortle,8,10,225,water,None,59,63,80,65,80,58
...,...,...,...,...,...,...,...,...,...,...,...,...
885,drakloak,886,14,110,dragon,ghost,68,80,50,60,50,102
886,dragapult,887,30,500,dragon,ghost,88,120,75,100,75,142
887,zacian,888,28,1100,fairy,None,92,130,115,80,115,138
888,zamazenta,889,29,2100,fighting,None,92,130,115,80,115,138


In [23]:
df_csv = pd.read_csv('Pokemon_full.csv', delimiter='\t', header=0)

In [21]:
print(df_csv.head())

         name  pokedex id  height  weight   type secundary type  hp  attack  \
0   bulbasaur           1       7      69  grass         poison  45      49   
1     ivysaur           2      10     130  grass         poison  60      62   
2    venusaur           3      20    1000  grass         poison  80      82   
3  charmander           4       6      85   fire           None  39      52   
4  charmeleon           5      11     190   fire           None  58      64   

   defense  sp atk  sp def  speed  
0       49      65      65     45  
1       63      80      80     60  
2       83     100     100     80  
3       43      60      50     65  
4       58      80      65     80  


In [29]:
len(df_csv)

890


# API request

In [2]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests as rq

In [7]:
base_url = 'https://www.boredapi.com/api/activity'

In [8]:
# payload = {'participants':1}

In [18]:
requests = rq.get(base_url)

In [19]:
data = requests.json()

In [20]:
print('Response:', data)

Response: {'activity': 'Memorize the fifty states and their capitals', 'type': 'education', 'participants': 1, 'price': 0, 'link': '', 'key': '4179309', 'accessibility': 0}


In [21]:
import requests

url = 'https://www.boredapi.com/api/activity'
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")


{'activity': 'Pick up litter around your favorite park', 'type': 'charity', 'participants': 1, 'price': 0, 'link': '', 'key': '4894697', 'accessibility': 0.05}
